In [1]:
pip install pandas numpy holidays scikit-learn openpyxl xlrd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.3/932.3 kB 8.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 3.4 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


# 1) Pipeline pandas: ingesta, limpieza y exportación de datos

In [12]:
# ------------------ Imports ------------------
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import StandardScaler

# ------------------ Parâmetros ------------------
file_path = "Contugas_Datos.xlsx"
output_path = "data_cleaned.csv"

# ------------------ 1. Ingestão de dados ------------------
excel = pd.ExcelFile(file_path)
lista_dfs = []

for sheet in excel.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df['Cliente'] = sheet
    lista_dfs.append(df)

data = pd.concat(lista_dfs, ignore_index=True)
print(f"[1] Ingestão: {len(data)} linhas carregadas")

# ------------------ 2. Pré-processamento ------------------
# Seleção de colunas
data = data[['Fecha', 'Presion', 'Temperatura', 'Volumen', 'Cliente']].copy()

# Conversão de data e remoção de nulos
data['Fecha'] = pd.to_datetime(data['Fecha'], errors='coerce')
data.dropna(inplace=True)

# Extração de mês e ano
data['Mes'] = data['Fecha'].dt.month
data['Ano'] = data['Fecha'].dt.year
print(f"[2] Pré-processamento: colunas atuais {list(data.columns)}")

# ------------------ 3. Suavização da variabilidade ------------------
window = 7  # ajustável
data['Presion_Suavizada'] = data['Presion'].rolling(window=window, center=True).mean()
# remover colunas antigas se existirem
data.drop(['Presion_Suavizada_15', 'Presion_Suavizada_30'], axis=1, errors='ignore', inplace=True)
print(f"[3] Suavização: média móvel ({window} dias) aplicada")

# ------------------ 4. Adição de feriados ------------------
anos = data['Ano'].unique().tolist()
feriados_pe = holidays.Peru(years=anos)
datas_feriados = {d.strftime("%Y-%m-%d") for d in feriados_pe.keys()}
data['Es_Feriado'] = data['Fecha'].dt.strftime("%Y-%m-%d").isin(datas_feriados)
print("[4] Feriados: coluna Es_Feriado criada")

# ------------------ 5. Categorização de volume por cliente ------------------
for cliente in data['Cliente'].unique():
    sub = data[data['Cliente'] == cliente]
    p10, p90 = sub['Volumen'].quantile(0.10), sub['Volumen'].quantile(0.90)
    mask = data['Cliente'] == cliente
    data.loc[mask, 'Anomalia'] = sub['Volumen'].apply(
        lambda x: 'Anômalo' if x < p10 or x > p90 else 'Normal'
    )
data['Anomalia_bin'] = data['Anomalia'].map({'Anômalo': 1, 'Normal': 0})
print("[5] Anomalias: colunas Anomalia e Anomalia_bin criadas")

# ------------------ 6. Filtragem de colunas finais ------------------
col_final = [
    'Presion', 'Temperatura', 'Volumen', 'Cliente',
    'Mes', 'Ano', 'Es_Feriado', 'Anomalia', 'Anomalia_bin'
]
data = data[col_final].copy()
print(f"[6] Colunas finais: {col_final}")

# ------------------ 7. Salvamento ------------------
data.to_csv(output_path, index=False)
print(f"[7] Salvamento: dados salvos em '{output_path}'")

[1] Ingestão: 847960 linhas carregadas
[2] Pré-processamento: colunas atuais ['Fecha', 'Presion', 'Temperatura', 'Volumen', 'Cliente', 'Mes', 'Ano']
[3] Suavização: média móvel (7 dias) aplicada
[4] Feriados: coluna Es_Feriado criada
[5] Anomalias: colunas Anomalia e Anomalia_bin criadas
[6] Colunas finais: ['Presion', 'Temperatura', 'Volumen', 'Cliente', 'Mes', 'Ano', 'Es_Feriado', 'Anomalia', 'Anomalia_bin']
[7] Salvamento: dados salvos em 'data_cleaned.csv'


# 2) Validación de datos Generados con Pandas por medio de Spark SQL (Pyspark)

In [19]:
from pyspark.sql import SparkSession

# 1. Criar SparkSession
spark = SparkSession.builder \
    .appName("ContugasTest") \
    .getOrCreate()

# 2. Ler o CSV gerado
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("data_cleaned.csv")  # ajuste o caminho se estiver em S3: "s3://meu-bucket/data_cleaned.csv"

# 3. Criar view temporária (nome arbitrário)
df.createOrReplaceTempView("contugas_pandas_procedado")

In [20]:
# Consulta 1: Mostrar las primeras 10 filas de la tabla
spark.sql("""
    SELECT *
    FROM contugas_pandas_procedado
    LIMIT 10
""").show(10, truncate=False)

+-----------------+-----------------+-----------------+--------+---+----+----------+--------+------------+
|Presion          |Temperatura      |Volumen          |Cliente |Mes|Ano |Es_Feriado|Anomalia|Anomalia_bin|
+-----------------+-----------------+-----------------+--------+---+----+----------+--------+------------+
|17.7325634924889 |28.2093536541928 |20.96975076975659|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.74777603806793|28.51861421312152|17.84573913758869|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.75891638774564|28.23019056507057|20.97591383522386|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.72794022684193|27.8115085858999 |20.59229909002912|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.74648374408114|27.79529343321371|21.69062581523098|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.73554042397516|27.66645651943054|19.09171697331939|CLIENTE1|1  |2019|false     |Normal  |0           |
|17.73179953287394|27.53234547004581|

In [3]:
# Consulta 2: Contar el número total de registros por tipo de anomalía
spark.sql("""
    SELECT Anomalia, COUNT(*) AS total
    FROM contugas_pandas_procedado
    GROUP BY Anomalia
    ORDER BY Anomalia
""").show(truncate=False)

+--------+------+
|Anomalia|total |
+--------+------+
|Anômalo |131750|
|Normal  |716210|
+--------+------+



In [4]:
# Consulta 3: Contar registros agrupados por cliente y tipo de anomalía
spark.sql("""
    SELECT Cliente, Anomalia, COUNT(*) AS total
    FROM contugas_pandas_procedado
    GROUP BY Cliente, Anomalia
    ORDER BY Cliente, Anomalia
""").show(truncate=False)

+---------+--------+-----+
|Cliente  |Anomalia|total|
+---------+--------+-----+
|CLIENTE1 |Anômalo |8684 |
|CLIENTE1 |Normal  |34728|
|CLIENTE10|Anômalo |4106 |
|CLIENTE10|Normal  |36953|
|CLIENTE11|Anômalo |8450 |
|CLIENTE11|Normal  |33798|
|CLIENTE12|Anômalo |4178 |
|CLIENTE12|Normal  |37598|
|CLIENTE13|Anômalo |4106 |
|CLIENTE13|Normal  |36953|
|CLIENTE14|Anômalo |8684 |
|CLIENTE14|Normal  |34731|
|CLIENTE15|Anômalo |4243 |
|CLIENTE15|Normal  |38185|
|CLIENTE16|Anômalo |8630 |
|CLIENTE16|Normal  |34517|
|CLIENTE17|Anômalo |8684 |
|CLIENTE17|Normal  |34728|
|CLIENTE18|Anômalo |8278 |
|CLIENTE18|Normal  |33104|
+---------+--------+-----+
only showing top 20 rows



In [5]:
# Consulta 4: Verificar el total de filas, clientes distintos y periodos distintos
spark.sql("""
    SELECT
      COUNT(*) AS total_linhas,
      COUNT(DISTINCT Cliente) AS clientes_distintos,
      COUNT(DISTINCT CONCAT(Ano,'-',Mes)) AS periodos_distintos
    FROM contugas_pandas_procedado
""").show()

+------------+------------------+------------------+
|total_linhas|clientes_distintos|periodos_distintos|
+------------+------------------+------------------+
|      847960|                20|                60|
+------------+------------------+------------------+



In [6]:
# Consulta 5: Distribución de anomalia_bin y conteo total por cada valor
spark.sql("""
    SELECT Anomalia_bin, COUNT(*) AS total
    FROM contugas_pandas_procedado
    GROUP BY Anomalia_bin
    ORDER BY Anomalia_bin
""").show(truncate=False)

+------------+------+
|Anomalia_bin|total |
+------------+------+
|0           |716210|
|1           |131750|
+------------+------+



In [7]:
# Consulta 6: Comprobar consistencia entre Anomalia y Anomalia_bin
spark.sql("""
    SELECT Anomalia, Anomalia_bin, COUNT(*) AS total
    FROM contugas_pandas_procedado
    GROUP BY Anomalia, Anomalia_bin
    ORDER BY Anomalia, Anomalia_bin
""").show(truncate=False)

+--------+------------+------+
|Anomalia|Anomalia_bin|total |
+--------+------------+------+
|Anômalo |1           |131750|
|Normal  |0           |716210|
+--------+------------+------+



In [8]:
# Consulta 7: Obtener estadísticas (promedio, mínimo, máximo) de Presion, Temperatura y Volumen
spark.sql("""
    SELECT
      ROUND(AVG(Presion),2)     AS avg_presion,
      ROUND(MIN(Presion),2)     AS min_presion,
      ROUND(MAX(Presion),2)     AS max_presion,
      ROUND(AVG(Temperatura),2) AS avg_temp,
      ROUND(MIN(Temperatura),2) AS min_temp,
      ROUND(MAX(Temperatura),2) AS max_temp,
      ROUND(AVG(Volumen),2)     AS avg_vol,
      ROUND(MIN(Volumen),2)     AS min_vol,
      ROUND(MAX(Volumen),2)     AS max_vol
    FROM contugas_pandas_procedado
""").show(truncate=False)

+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+
|avg_presion|min_presion|max_presion|avg_temp|min_temp|max_temp|avg_vol|min_vol|max_vol|
+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+
|16.07      |2.93       |20.31      |25.2    |-5.26   |50.02   |62.33  |0.0    |577.41 |
+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+



In [9]:
# Consulta 8: Calcular el porcentaje de anomalías por cliente
spark.sql("""
    SELECT
      Cliente,
      ROUND(100 * SUM(Anomalia_bin) / COUNT(*),2) AS pct_anomalia
    FROM contugas_pandas_procedado
    GROUP BY Cliente
    ORDER BY Cliente
""").show(truncate=False)

+---------+------------+
|Cliente  |pct_anomalia|
+---------+------------+
|CLIENTE1 |20.0        |
|CLIENTE10|10.0        |
|CLIENTE11|20.0        |
|CLIENTE12|10.0        |
|CLIENTE13|10.0        |
|CLIENTE14|20.0        |
|CLIENTE15|10.0        |
|CLIENTE16|20.0        |
|CLIENTE17|20.0        |
|CLIENTE18|20.0        |
|CLIENTE19|10.0        |
|CLIENTE2 |20.0        |
|CLIENTE20|20.0        |
|CLIENTE3 |20.0        |
|CLIENTE4 |10.0        |
|CLIENTE5 |10.0        |
|CLIENTE6 |20.0        |
|CLIENTE7 |10.0        |
|CLIENTE8 |20.0        |
|CLIENTE9 |10.0        |
+---------+------------+



In [10]:
# Consulta 9: Evolución mensual de anomalías (total y porcentaje)
spark.sql("""
    SELECT
      Ano,
      Mes,
      COUNT(*) AS total,
      ROUND(100 * SUM(Anomalia_bin) / COUNT(*),2) AS pct_anomalia
    FROM contugas_pandas_procedado
    GROUP BY Ano, Mes
    ORDER BY Ano, Mes
""").show(truncate=False)

+----+---+-----+------------+
|Ano |Mes|total|pct_anomalia|
+----+---+-----+------------+
|2019|1  |8622 |19.37       |
|2019|2  |13304|16.84       |
|2019|3  |14530|17.89       |
|2019|4  |12328|12.41       |
|2019|5  |14816|11.43       |
|2019|6  |14166|12.35       |
|2019|7  |14730|13.05       |
|2019|8  |14612|11.15       |
|2019|9  |12990|12.16       |
|2019|10 |13330|12.61       |
|2019|11 |14288|12.84       |
|2019|12 |14754|13.43       |
|2020|1  |14810|19.01       |
|2020|2  |13728|16.79       |
|2020|3  |14766|17.21       |
|2020|4  |14342|17.81       |
|2020|5  |14774|16.47       |
|2020|6  |14194|14.32       |
|2020|7  |14602|11.37       |
|2020|8  |14590|11.8        |
+----+---+-----+------------+
only showing top 20 rows



In [11]:
# Consulta 10: Distribución entre días festivos y días laborables con porcentaje sobre el total
spark.sql("""
    SELECT
      Es_Feriado,
      COUNT(*) AS total,
      ROUND(100 * COUNT(*) / (SELECT COUNT(*) FROM contugas_trusted_dato_procesado),2) AS pct
    FROM contugas_pandas_procedado
    GROUP BY Es_Feriado
    ORDER BY Es_Feriado
""").show(truncate=False)

+----------+------+-----+
|Es_Feriado|total |pct  |
+----------+------+-----+
|false     |816302|96.27|
|true      |31658 |3.73 |
+----------+------+-----+



# 3) Validación de datos Generados con AWS Glue/Pyspark por medio de Spark SQL (Pyspark)

Estos archivos que se leerán a continuación fueron extraídos del bucket de la capa trusted del data lake para validar los datos, es decir, para comprobar si lo que se generó con Glue y PySpark coincide con lo que se había hecho inicialmente con pandas.

In [26]:
df_refined = (
    spark.read
         .parquet("Parquet_Trusted_Files/part-*.snappy.parquet")
         .withColumnRenamed("año", "ano")
)

df_refined.createOrReplaceTempView("contugas_trusted_dato_procesado")

In [27]:
# Consulta 1: Mostrar las primeras 10 filas de la tabla
spark.sql("""
    SELECT *
    FROM contugas_trusted_dato_procesado
    LIMIT 10
""").show(10, truncate=False)

+---------+-------------------+-----------------+-----------------+-----------------+---+----+------------------+----------+------------------+------------------+--------+------------+
|cliente  |fecha              |presion          |temperatura      |volumen          |mes|ano |presion_suavizada |es_feriado|p10               |p90               |anomalia|anomalia_bin|
+---------+-------------------+-----------------+-----------------+-----------------+---+----+------------------+----------+------------------+------------------+--------+------------+
|CLIENTE17|2019-01-19 00:00:00|17.77212441594732|27.85500379522216|23.68536560418588|1  |2019|17.533307762226894|false     |11.290658237352273|26.359754860759395|Normal  |0           |
|CLIENTE17|2019-01-19 01:00:00|17.78868605831389|27.98359695831391|22.82341722448241|1  |2019|17.59235216149999 |false     |11.290658237352273|26.359754860759395|Normal  |0           |
|CLIENTE17|2019-01-19 02:00:00|17.85616205512526|27.94465807933698|20.15624

In [28]:
# Consulta 2: Contar el número total de registros por tipo de anomalía
spark.sql("""
    SELECT Anomalia, COUNT(*) AS total
    FROM contugas_trusted_dato_procesado
    GROUP BY Anomalia
    ORDER BY Anomalia
""").show(truncate=False)

+--------+------+
|Anomalia|total |
+--------+------+
|Anómalo |131750|
|Normal  |716210|
+--------+------+



In [29]:
# Consulta 3: Contar registros agrupados por cliente y tipo de anomalía
spark.sql("""
    SELECT Cliente, Anomalia, COUNT(*) AS total
    FROM contugas_trusted_dato_procesado
    GROUP BY Cliente, Anomalia
    ORDER BY Cliente, Anomalia
""").show(truncate=False)

+---------+--------+-----+
|Cliente  |Anomalia|total|
+---------+--------+-----+
|CLIENTE1 |Anómalo |8684 |
|CLIENTE1 |Normal  |34728|
|CLIENTE10|Anómalo |4106 |
|CLIENTE10|Normal  |36953|
|CLIENTE11|Anómalo |8450 |
|CLIENTE11|Normal  |33798|
|CLIENTE12|Anómalo |4178 |
|CLIENTE12|Normal  |37598|
|CLIENTE13|Anómalo |4106 |
|CLIENTE13|Normal  |36953|
|CLIENTE14|Anómalo |8684 |
|CLIENTE14|Normal  |34731|
|CLIENTE15|Anómalo |4243 |
|CLIENTE15|Normal  |38185|
|CLIENTE16|Anómalo |8630 |
|CLIENTE16|Normal  |34517|
|CLIENTE17|Anómalo |8684 |
|CLIENTE17|Normal  |34728|
|CLIENTE18|Anómalo |8278 |
|CLIENTE18|Normal  |33104|
+---------+--------+-----+
only showing top 20 rows



In [30]:
# Consulta 4: Verificar el total de filas, clientes distintos y periodos distintos
spark.sql("""
    SELECT
      COUNT(*) AS total_linhas,
      COUNT(DISTINCT Cliente) AS clientes_distintos,
      COUNT(DISTINCT CONCAT(Ano,'-',Mes)) AS periodos_distintos
    FROM contugas_trusted_dato_procesado
""").show()

+------------+------------------+------------------+
|total_linhas|clientes_distintos|periodos_distintos|
+------------+------------------+------------------+
|      847960|                20|                60|
+------------+------------------+------------------+



In [31]:
# Consulta 5: Distribución de anomalia_bin y conteo total por cada valor
spark.sql("""
    SELECT Anomalia_bin, COUNT(*) AS total
    FROM contugas_trusted_dato_procesado
    GROUP BY Anomalia_bin
    ORDER BY Anomalia_bin
""").show(truncate=False)

+------------+------+
|Anomalia_bin|total |
+------------+------+
|0           |716210|
|1           |131750|
+------------+------+



In [32]:
# Consulta 6: Comprobar consistencia entre Anomalia y Anomalia_bin
spark.sql("""
    SELECT Anomalia, Anomalia_bin, COUNT(*) AS total
    FROM contugas_trusted_dato_procesado
    GROUP BY Anomalia, Anomalia_bin
    ORDER BY Anomalia, Anomalia_bin
""").show(truncate=False)

+--------+------------+------+
|Anomalia|Anomalia_bin|total |
+--------+------------+------+
|Anómalo |1           |131750|
|Normal  |0           |716210|
+--------+------------+------+



In [33]:
# Consulta 7: Obtener estadísticas (promedio, mínimo, máximo) de Presion, Temperatura y Volumen
spark.sql("""
    SELECT
      ROUND(AVG(Presion),2)     AS avg_presion,
      ROUND(MIN(Presion),2)     AS min_presion,
      ROUND(MAX(Presion),2)     AS max_presion,
      ROUND(AVG(Temperatura),2) AS avg_temp,
      ROUND(MIN(Temperatura),2) AS min_temp,
      ROUND(MAX(Temperatura),2) AS max_temp,
      ROUND(AVG(Volumen),2)     AS avg_vol,
      ROUND(MIN(Volumen),2)     AS min_vol,
      ROUND(MAX(Volumen),2)     AS max_vol
    FROM contugas_trusted_dato_procesado
""").show(truncate=False)

+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+
|avg_presion|min_presion|max_presion|avg_temp|min_temp|max_temp|avg_vol|min_vol|max_vol|
+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+
|16.07      |2.93       |20.31      |25.2    |-5.26   |50.02   |62.33  |0.0    |577.41 |
+-----------+-----------+-----------+--------+--------+--------+-------+-------+-------+



In [34]:
# Consulta 8: Calcular el porcentaje de anomalías por cliente
spark.sql("""
    SELECT
      Cliente,
      ROUND(100 * SUM(Anomalia_bin) / COUNT(*),2) AS pct_anomalia
    FROM contugas_trusted_dato_procesado
    GROUP BY Cliente
    ORDER BY Cliente
""").show(truncate=False)

+---------+------------+
|Cliente  |pct_anomalia|
+---------+------------+
|CLIENTE1 |20.0        |
|CLIENTE10|10.0        |
|CLIENTE11|20.0        |
|CLIENTE12|10.0        |
|CLIENTE13|10.0        |
|CLIENTE14|20.0        |
|CLIENTE15|10.0        |
|CLIENTE16|20.0        |
|CLIENTE17|20.0        |
|CLIENTE18|20.0        |
|CLIENTE19|10.0        |
|CLIENTE2 |20.0        |
|CLIENTE20|20.0        |
|CLIENTE3 |20.0        |
|CLIENTE4 |10.0        |
|CLIENTE5 |10.0        |
|CLIENTE6 |20.0        |
|CLIENTE7 |10.0        |
|CLIENTE8 |20.0        |
|CLIENTE9 |10.0        |
+---------+------------+



In [36]:
# Consulta 9: Evolución mensual de anomalías (total y porcentaje)
spark.sql("""
    SELECT
      Ano,
      Mes,
      COUNT(*) AS total,
      ROUND(100 * SUM(Anomalia_bin) / COUNT(*),2) AS pct_anomalia
    FROM contugas_trusted_dato_procesado
    GROUP BY Ano, Mes
    ORDER BY Ano, Mes
""").show(truncate=False)

+----+---+-----+------------+
|Ano |Mes|total|pct_anomalia|
+----+---+-----+------------+
|2019|1  |8622 |19.37       |
|2019|2  |13304|16.84       |
|2019|3  |14530|17.89       |
|2019|4  |12328|12.41       |
|2019|5  |14816|11.43       |
|2019|6  |14166|12.35       |
|2019|7  |14730|13.05       |
|2019|8  |14612|11.15       |
|2019|9  |12990|12.16       |
|2019|10 |13330|12.61       |
|2019|11 |14288|12.84       |
|2019|12 |14754|13.43       |
|2020|1  |14810|19.01       |
|2020|2  |13728|16.79       |
|2020|3  |14766|17.21       |
|2020|4  |14342|17.81       |
|2020|5  |14774|16.47       |
|2020|6  |14194|14.32       |
|2020|7  |14602|11.37       |
|2020|8  |14590|11.8        |
+----+---+-----+------------+
only showing top 20 rows



In [37]:
# Consulta 10: Distribución entre días festivos y días laborables con porcentaje sobre el total
spark.sql("""
    SELECT
      Es_Feriado,
      COUNT(*) AS total,
      ROUND(100 * COUNT(*) / (SELECT COUNT(*) FROM contugas_trusted_dato_procesado),2) AS pct
    FROM contugas_trusted_dato_procesado
    GROUP BY Es_Feriado
    ORDER BY Es_Feriado
""").show(truncate=False)

+----------+------+-----+
|Es_Feriado|total |pct  |
+----------+------+-----+
|false     |816302|96.27|
|true      |31658 |3.73 |
+----------+------+-----+

